In [ ]:
import pandas as pd
import numpy as np
import functionsML as f
from sklearn.model_selection import train_test_split, PredefinedSplit, RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import randint
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [ ]:
# Função de Limpeza Básica (Sem Leakage)
def clean_data(df):
    df = df.copy()
    
    # Drop irrelevant
    df = df.drop(columns=["hasDamage","paintQuality%"], errors='ignore')
    
    # Text handling
    text_cols = df.select_dtypes(include=["object"]).columns
    df[text_cols] = df[text_cols].apply(lambda x: x.str.lower() if x.dtype=="object" else x)
    for col in df.select_dtypes(include="object").columns:
        df = f.fix_typos(col, df)

    # Transmission: 'other' passa a 'unknown'
    df['transmission'] = df['transmission'].replace('other', 'unknown')
    
    # FuelType: 'other' passa a 'electric' (conforme o teu código antigo)
    df['fuelType'] = df['fuelType'].replace('other', 'electric')

    # Filtering / Cleaning Rules
    df.loc[df["mileage"] < 0, "mileage"] = np.nan
    df.loc[~df["tax"].between(0, 600), "tax"] = np.nan
    df.loc[~df["mpg"].between(0, 150), "mpg"] = np.nan
    df.loc[~df["engineSize"].between(1, 6.3), "engineSize"] = np.nan
    df.loc[~df["year"].between(1990, 2020), "year"] = np.nan
    df.loc[~df["previousOwners"].between(0, 6), "previousOwners"] = np.nan # Opcional conforme o teu código

    # Numeric Transformations
    df['mileage'] = np.log1p(df['mileage'])
    df['mpg'] = np.log1p(df['mpg'])
    df['tax'] = np.log1p(df['tax'])
    
    # Types and Rounding
    df["year"] = df["year"].round()
    df["previousOwners"] = pd.to_numeric(df["previousOwners"], errors='coerce').round().astype("Int64")
    df["year"] = pd.to_numeric(df["year"], errors='coerce').round().astype("Int64")
    
    # Imputation
    df = f.fill_NaN_with_categorical(df, "Brand", ["model","transmission","fuelType"])
    df = f.fill_NaN_with_categorical(df, "Brand", ["model","transmission"])
    df = f.fill_NaN_with_categorical(df, "model", ["Brand","transmission","fuelType"])
    df = f.fill_NaN_with_categorical(df, "model", ["Brand","transmission"])
    df = f.fill_NaN_with_categorical(df, "mpg", ["model","fuelType"])
    
    df["transmission"] = df["transmission"].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else np.nan))
    df["fuelType"] = df["fuelType"].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else np.nan))

    df = f.fill_NaN_with_mixed(df, "year", "model", "mileage")
    df = f.fill_NaN_with_mixed(df, "mileage", "model", "year")
    df = f.fill_NaN_with_mixed(df, "tax", "model", "year")
    df = f.fill_NaN_with_mixed(df, "engineSize", "model", "tax")

    df["previousOwners"] = df["previousOwners"].transform(lambda x: x.fillna(x.median())).round().astype("Int64")
    
    # Residual Fill
    numeric_cols = df.select_dtypes(include=["number"]).columns.drop(["carID", "price"], errors='ignore')
    for col in numeric_cols:
        df[col] = df[col].astype(float)
        global_mean = df[col].median()
        df[col] = df[col].fillna(global_mean)
        if "Int64" in str(df[col].dtype):
            df[col] = df[col].round().astype("Int64")
            
    return df

In [ ]:
# 1. Carregar Dados
train_db = pd.read_csv("./train.csv")
# Importante: Aplicar log no target logo no início se for essa a estratégia
train_db['price'] = np.log1p(train_db['price'])

# 2. HOLDOUT SPLIT (Separação Inicial)
# Aqui garantimos que a validação nunca vê o treino
train_set, val_set = train_test_split(train_db, test_size=0.3, random_state=42, shuffle=True)

# 3. Limpeza Independente (Clean Data)
train_set = clean_data(train_set)
val_set = clean_data(val_set)

# 4. ENCODING & SCALING (Onde ocorre o Data Leakage se não tiver cuidado)

# A. One-Hot Encoding
# Temos de garantir colunas iguais. Concatenamos só para gerar as dummies e separamos de novo.
train_len = len(train_set)
combined_temp = pd.concat([train_set, val_set], axis=0)

combined_temp = pd.get_dummies(combined_temp, columns=["Brand", "transmission", "fuelType"], drop_first=True)

# Separar de volta
train_set_encoded = combined_temp.iloc[:train_len].copy()
val_set_encoded = combined_temp.iloc[train_len:].copy()

# B. Target Encoding (FIT no Treino, TRANSFORM no Treino e Validação)
# Calcular médias no Treino
mapping = train_set.groupby(["Brand", "model"])["price"].mean().to_dict()
global_mean = train_set["price"].mean()

# Aplicar ao Treino
train_set_encoded["Brand_model_encoded"] = train_set.apply(
    lambda x: mapping.get((x["Brand"], x["model"]), global_mean), axis=1
)

# Aplicar à Validação (usando o mapping do treino!)
val_set_encoded["Brand_model_encoded"] = val_set.apply(
    lambda x: mapping.get((x["Brand"], x["model"]), global_mean), axis=1
)

# 5. PREPARAÇÃO FINAL (X e y)
drop_cols = ["price", "carID", "model", "previousOwners"] # Colunas a remover

X_train = train_set_encoded.drop(columns=drop_cols, errors='ignore')
y_train = train_set_encoded["price"]

X_val = val_set_encoded.drop(columns=drop_cols, errors='ignore')
y_val = val_set_encoded["price"]

# C. Scaling (FIT no Treino, TRANSFORM em ambos)
scaler = MinMaxScaler()
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns, index=X_train.index)
X_val_scaled = pd.DataFrame(scaler.transform(X_val), columns=X_val.columns, index=X_val.index)

In [ ]:
# 1. Juntar os dados processados (Reset index para garantir alinhamento)
X_combined = pd.concat([X_train_scaled, X_val_scaled], axis=0).reset_index(drop=True)
y_combined = pd.concat([y_train, y_val], axis=0).reset_index(drop=True)

# 2. Criar a "Máscara" de Split (test_fold)
# -1 indica: "Este dado é de treino, usa para aprender"
#  0 indica: "Este dado é de validação, usa para testar" (0 é o índice do fold de validação)

# Array com -1 para o tamanho do treino
split_index_train = [-1] * len(X_train_scaled)
# Array com 0 para o tamanho da validação
split_index_val = [0] * len(X_val_scaled)

# Juntar os dois
test_fold = split_index_train + split_index_val

# 3. Criar o Objeto PredefinedSplit
ps = PredefinedSplit(test_fold)

In [ ]:
# 4. Configurar o RandomizedSearchCV com cv=ps
# Exemplo com Random Forest
rf_model = RandomForestRegressor(random_state=42, n_jobs=-1)

param_dist = {
    'n_estimators': randint(50, 300),
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': randint(2, 20),
    'min_samples_leaf': randint(1, 10)
}

random_search = RandomizedSearchCV(
    estimator=rf_model,
    param_distributions=param_dist,
    n_iter=50,    # Número de tentativas
    cv=ps,        # <--- AQUI ESTÁ O TRUQUE: Usamos o nosso split personalizado
    scoring='neg_root_mean_squared_error',
    verbose=1,
    n_jobs=-1,
    random_state=42
)

# 5. Executar a Busca
print("A iniciar Random Search com Predefined Split...")
# Passamos o X_combined e y_combined. Ele vai saber separar internamente graças ao 'ps'.
random_search.fit(X_combined, y_combined)

# 6. Resultados
print(f"Melhores Parâmetros: {random_search.best_params_}")
print(f"Melhor RMSE (log): {-random_search.best_score_:.4f}")

# O melhor modelo já fica treinado com TUDO (Treino + Validação) se refit=True (default)
best_model = random_search.best_estimator_

In [ ]:
# --- AVALIAÇÃO DO MODELO NA VALIDAÇÃO ---

# 1. Fazer Previsões
# Certifica-te que 'best_model' é o nome do teu modelo treinado (ou best_knn, best_rf, etc.)
print("A calcular previsões na validação...")
y_pred_val_log = best_model.predict(X_val_scaled)

# 2. Inverter o Logaritmo (Voltar para Euros)
# Como treinaste com np.log1p, tens de usar np.expm1 para reverter
y_pred_val_real = np.expm1(y_pred_val_log)
y_val_real = np.expm1(y_val) # O y_val original também estava em log

# 3. Calcular Métricas
r2 = r2_score(y_val_real, y_pred_val_real)
mae = mean_absolute_error(y_val_real, y_pred_val_real)
rmse = np.sqrt(mean_squared_error(y_val_real, y_pred_val_real))

print(f"R² Score: {r2:.4f}  (Ideal: 1.0)")
print(f"MAE:      {mae:.2f} €  (Erro médio em Euros)")
print(f"RMSE:     {rmse:.2f} €  (Erro penaliza outliers)")

In [ ]:
# 1. Carregar Teste
test_db = pd.read_csv("./test.csv")

# 2. Aplicar a mesma Limpeza (Função que criámos antes)
# Nota: Garante que definiste a função 'clean_data' no bloco anterior
test_db = clean_data(test_db) 

# 3. ENCODING (Cuidado Máximo aqui!)

# A. One-Hot Encoding
# O get_dummies no teste pode gerar colunas diferentes. Vamos resolver isso no passo de alinhamento.
test_db_encoded = pd.get_dummies(test_db, columns=["Brand", "transmission", "fuelType"], drop_first=True)

# B. Target Encoding
# IMPORTANTE: Usar o 'mapping' e 'global_mean' que calculaste no TREINO (não recalcules no teste!)
if 'mapping' not in locals() or 'global_mean' not in locals():
    raise ValueError("Erro: As variáveis 'mapping' e 'global_mean' do treino não estão na memória.")

test_db_encoded["Brand_model_encoded"] = test_db.apply(
    lambda x: mapping.get((x["Brand"], x["model"]), global_mean), axis=1
)

# 4. PREPARAÇÃO FINAL
# Remover as mesmas colunas que removemos no treino
drop_cols = ["price", "carID", "model", "previousOwners"]
X_test = test_db_encoded.drop(columns=drop_cols, errors='ignore')

# 5. ALINHAMENTO DE COLUNAS (A "Vacina" contra erros)
# O modelo foi treinado com X_combined. O X_test tem de ter exatamente as mesmas colunas.
# Se faltar alguma (ex: Ferrari), preenchemos com 0. Se houver a mais, ignoramos.
cols_treino = X_combined.columns # Colunas usadas no fit do RandomizedSearch
X_test = X_test.reindex(columns=cols_treino, fill_value=0)

# 6. SCALING
# Usar o 'scaler' já treinado (não fazer fit!)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

# 7. PREVISÃO
# Usar o best_model (que já está treinado com tudo)
print("A fazer previsões finais...")
y_test_pred_log = best_model.predict(X_test_scaled)

# 8. INVERTER O LOG (Trazer de volta para Euros)
y_test_pred_real = np.expm1(y_test_pred_log)

# 9. GUARDAR SUBMISSÃO
submission = pd.DataFrame({
    "carID": test_db["carID"], # Buscar o ID original
    "price": y_test_pred_real
})

submission.to_csv("submission_final_campeao.csv", index=False)
print("Sucesso! Ficheiro 'submission_final_campeao.csv' criado.")